In [85]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import AdamW

In [86]:
train_dir = '/home/research/Kilsar_Sentinal/EfficientNet data/splitted_data/train'
val_dir = '/home/research/Kilsar_Sentinal/EfficientNet data/splitted_data/val'

In [87]:
# General Hyperparameters
input_shape = (300, 300, 3)         
num_classes = 2                       
dropout_rate = 0.4                    

# Model Architecture Hyperparameters
dense_units_1 = 256                 
dense_units_2 = 128                 

# Data Augmentation Hyperparameters
rescale_value = 1./255             
rotation_range = 20                 
width_shift_range = 0.2
height_shift_range = 0.2              
shear_range = 0.2                  
zoom_range = 0.2                     
horizontal_flip = True            
fill_mode = 'nearest'              

# Training Hyperparameters
batch_size = 15                      
epochs = 10                               

# Callbacks Hyperparameters
reduce_lr_factor = 0.2             
reduce_lr_patience = 3            
min_lr = 1e-6                     
early_stopping_patience = 5      

# optimizerAdam = 'adam'
optimizerAdam = AdamW(
    learning_rate=0.005,
    weight_decay=0.004,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name='adamw',
)


In [88]:
base_model = EfficientNetB3(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape,
    pooling='avg'
)

model = models.Sequential([
    base_model,
    layers.Dense(dense_units_1, activation='relu'),
    layers.Dense(dense_units_2, activation='relu'),
    layers.Dropout(dropout_rate),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=optimizerAdam,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ (None, 1536)           │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 256)            │       393,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,210,161 (42.76 MB)

 Trainable params: 11,122,858 (42.43 MB)

 Non-trainable params: 87,303 (341.03 KB)

In [89]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=rescale_value,
    rotation_range=rotation_range,
    width_shift_range=width_shift_range,
    height_shift_range=height_shift_range,
    shear_range=shear_range,
    zoom_range=zoom_range,
    horizontal_flip=horizontal_flip,
    fill_mode=fill_mode
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=rescale_value)

In [90]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]), 
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(input_shape[0], input_shape[1]), 
    batch_size=batch_size,
    class_mode='categorical'
)

Found 7996 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [91]:
# writing callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=reduce_lr_factor, patience=reduce_lr_patience, min_lr=min_lr)
early_stopping = EarlyStopping(monitor='val_loss', patience=early_stopping_patience, restore_best_weights=True)
callbacks = [reduce_lr, early_stopping]

In [93]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=callbacks
)

Epoch 1/10


2024-10-09 20:41:38.869468: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at xla_ops.cc:577 : UNKNOWN: Failed to determine best cudnn convolution algorithm for:
%cudnn-conv.569 = (f32[15,144,151,151]{3,2,1,0}, u8[0]{0}) custom-call(f32[15,144,153,153]{3,2,1,0} %pad.27, f32[144,1,3,3]{3,2,1,0} %transpose.1718), window={size=3x3}, dim_labels=bf01_oi01->bf01, feature_group_count=144, custom_call_target="__cudnn$convForward", metadata={op_type="DepthwiseConv2dNativeBackpropInput" op_name="gradient_tape/sequential_8_1/efficientnetb3_1/block2a_dwconv_1/depthwise/DepthwiseConv2dNativeBackpropInput" source_file="/home/research/Kilsar_Sentinal/.venv/lib64/python3.11/site-packages/tensorflow/python/framework/ops.py" source_line=1177}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}

Original error: RESOU

UnknownError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/research/Kilsar_Sentinal/.venv/lib64/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/research/Kilsar_Sentinal/.venv/lib64/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/research/Kilsar_Sentinal/.venv/lib64/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/research/Kilsar_Sentinal/.venv/lib64/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib64/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib64/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib64/python3.11/asyncio/events.py", line 84, in _run

  File "/home/research/Kilsar_Sentinal/

# Appy Grid Search

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define the function to build the model
def create_model(dense_units_1=256, dense_units_2=128, dropout_rate=0.2, learning_rate=0.001, input_shape=(300, 300, 3), num_classes=2):
    base_model = EfficientNetB3(include_top=False, weights='imagenet', input_shape=input_shape, pooling='avg')
    
    # Create a Sequential model and add layers
    model = models.Sequential([
        base_model,
        layers.Dense(dense_units_1, activation='relu'),
        layers.Dense(dense_units_2, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Wrap the model with KerasClassifier
model = KerasClassifier(model=create_model, verbose=1)

# Define the hyperparameter grid to search
param_grid = {
    'dense_units_1': [128, 256, 512],
    'dense_units_2': [64, 128, 256],
    'dropout_rate': [0.2, 0.3, 0.4],
    'learning_rate': [0.01, 0.001, 0.0001],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20]
}

# Create GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=3)

# Prepare your training data
train_dir = '/home/research/Kilsar_Sentinal/EfficientNet data/splitted_data/train'
val_dir = '/home/research/Kilsar_Sentinal/EfficientNet data/splitted_data/val'

# Create data generators
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

# Fit the GridSearch
try:
    grid_result = grid.fit(train_generator, verbose=1)
except Exception as e:
    print(f"Error during GridSearch fitting: {e}")

# Print the best hyperparameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


## Random Search

In [ ]:
import kerastuner as kt
from tensorflow.keras.optimizers import Adam

# enabling gpu
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth set successfully.")
    except RuntimeError as e:
        print(f"Error setting memory growth: {e}")


# Define a function for model building that integrates with Keras Tuner
def build_model(hp):
    base_model = EfficientNetB3(
        include_top=False,
        weights='imagenet',
        input_shape=(300, 300, 3),
        pooling='avg'
    )
    
    # Create the model
    model = models.Sequential([
        base_model,
        layers.Dense(hp.Int('dense_units_1', min_value=128, max_value=512, step=128), activation='relu'),
        layers.Dense(hp.Int('dense_units_2', min_value=64, max_value=256, step=64), activation='relu'),
        layers.Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.4, step=0.1)),
        layers.Dense(2, activation='softmax')
    ])
    
    # Compile the model
    model.compile(
        optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create a tuner instance
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='tuner_dir',
    project_name='hyperparam_tuning'
)

# Prepare training data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

# Perform the search
tuner.search(train_generator, epochs=10, validation_data=train_generator)

# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")


In [84]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  4
